In [14]:
def find_multiple_targets(data, targets):
    for i, sublist in enumerate(data):
        for target in targets:
            if target in sublist:
                return (i, sublist.index(target), target)  # Outer index, inner index, and the target
    return None  # If no target is found

# Example usage
data = [["apple", "banana"], ["carrot", "apple"], ["date", "fig", "grape"]]
targets = ["apple", "grape", "carrot"]

result = find_multiple_targets(data, targets)
print("Found at:", result)


Found at: (0, 0, 'apple')


In [4]:
import numpy as np
import pandas as pd
import re

In [ ]:
with open("2024\\Day_06\\test_data.txt", "r") as file:
        # Read each line, strip newline characters, and convert to a list of characters
        map = [list(line.strip()) for line in file.readlines()]

[['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '^', '.', '.', '.', '.', '#'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'], ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.']]


In [11]:
up, down, left, right = "^", "v", "<", ">"
for match in re.finditer(rf"{up}|{down}|{left}|{right}", str(grid)): position = match.group()
print(position)